In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Iskon.csv to Iskon.csv


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Iskon.csv']))
print(df)

      Unnamed: 0  REVIEW ID            SOURCE           REVIEW BY REVIEW DATE  \
0              0        814      Trip Advisor           maryd1928  02-01-2013   
1              2        813      Trip Advisor            shoubhik  05-01-2013   
2              3       1182      Trip Advisor            shoubhik  05-01-2013   
3              4        812      Trip Advisor  onthemoveBangalore  10-01-2013   
4              6        808      Trip Advisor      RobertBorchers  12-01-2013   
...          ...        ...               ...                 ...         ...   
4414        5677       5680  Google + HK HILL         Sudarshan S  10-08-2017   
4415        5678       5679  Google + HK HILL          Vignesh. O  10-08-2017   
4416        5680       5685      Trip Advisor           disheet p  11-08-2017   
4417        5683       5686      Trip Advisor       Lalengmawia V  12-08-2017   
4418        5684       5687      Trip Advisor        Dr_sumitarya  13-08-2017   

                      REVIE

In [4]:
#Change the dataframe name from 
df = df.rename(columns={'REVIEW TYPE': 'REVIEW_TYPE'})
df = df.rename(columns={'REVIEW SUBJECT': 'REVIEW_SUBJECT'})

In [5]:
# drop rows where review_type is 'QUERY' or 'SUGGESTION'
to_drop = df[(df['REVIEW_TYPE'] == 'QUERY') | (df['REVIEW_TYPE'] == 'SUGGESTION')].index
df = df.drop(to_drop)



In [6]:
df['text'] = df['text'].fillna('')
df['REVIEW_SUBJECT'] = df['REVIEW_SUBJECT'].fillna('')

In [7]:
# append REVIEW_SUBJECT to text column
df['text'] = df.apply(lambda row: row['text'] + ' ' + row['REVIEW_SUBJECT'], axis=1)

In [8]:
# https://towardsdatascience.com/translate-a-pandas-data-frame-using-googletrans-library-fb0aa7fca592
# install googletrans using pip
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=4e8e0512c292fa6c584d588d7370f75ae8a512286977d017ed525297135d2c3e
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans


In [9]:
from googletrans import Translator
translator = Translator()

In [10]:
df['translated_text'] = df['text'].apply(lambda x: translator.translate(x).text)


In [11]:
df.to_csv('Iskon_translated.csv')
files.download('Iskon_translated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# Converting MIXED to either POSITIVE or NEGATIVE based on the polarity
from textblob import TextBlob

def get_sentiment_polarity(text):
    '''
    Function to get the polarity score of the text using TextBlob
    Returns 'POSITIVE' if polarity > 0,
            'NEGATIVE' if polarity < 0,
            'NEUTRAL' if polarity = 0
    '''
    blob = TextBlob(str(text))
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return 'POSITIVE'
    elif polarity < 0:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

# Change the review_type column value to either POSITIVE or NEGATIVE based on the polarity score of the text column
df.loc[df['REVIEW_TYPE'] == 'MIXED', 'REVIEW_TYPE'] = df[df['REVIEW_TYPE'] == 'MIXED']['translated_text'].apply(get_sentiment_polarity)


In [14]:
from textblob import TextBlob

def get_sentiment_polarity(text):
    '''
    Function to get the polarity score of the text using TextBlob
    Returns 'POSITIVE' if polarity > 0.2,
            'NEGATIVE' if polarity < -0.2,
            'MIXED' if -0.2 <= polarity <= 0.2
    '''
    blob = TextBlob(str(text))
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return 'POSITIVE'
    elif polarity < 0:
        return 'NEGATIVE'
    else:
        return 'MIXED'

# Change the review_type column value to either POSITIVE or NEGATIVE based on the polarity score of the text column
df['REVIEW_TYPE'] = df['translated_text'].apply(get_sentiment_polarity)


In [15]:
df.to_csv('iskontranslated.csv')
from google.colab import files
files.download('iskontranslated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
df.to_csv('iskontranslated.csv')
from google.colab import files
files.download('iskontranslated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
df['translated_text'].fillna('', inplace=True)

In [18]:
import pandas as pd
from textblob import TextBlob

# perform sentiment analysis and categorize text as POSITIVE, NEGATIVE, or NEUTRAL
def get_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity >0.05:
        return 'POSITIVE'
    elif polarity <-0.05:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

df['sentiment'] = df['translated_text'].apply(get_sentiment)


In [19]:
# compare predicted sentiment with actual sentiment and calculate accuracy
correct = 0
total = len(df)
for index, row in df.iterrows():
    if row['sentiment'] == row['REVIEW_TYPE']:
        correct += 1

accuracy = correct / total
print('Accuracy:', accuracy)


Accuracy: 0.9108395564607378


In [20]:
# Assuming that you have a DataFrame called "df" with "sentiment" and "review_type" columns

def compare_sentiment_review_type(row):
    if row['sentiment'] != row['REVIEW_TYPE']:
        return 'different'
    else:
        return 'same'

df['sentiment_review_type_flag'] = df.apply(compare_sentiment_review_type, axis=1)


In [21]:
from google.colab import files
df.to_csv('sentiment_analysis.csv', index=False)
files.download('sentiment_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>